In [73]:
from preprocess import Preprocess
from urllib.request import urlopen
import timm
import torch
import functools
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [74]:
TRANSFORM = transforms.ToTensor()

In [75]:
class TIMM:
    def __init__(self):
        # https://huggingface.co/timm/vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k
        self.model = timm.create_model(
            'vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k',
            pretrained=True,
            num_classes=0,  # remove classifier nn.Linear
        )
        self.model.eval()

        # get model specific transforms (normalization, resize)
        data_config = timm.data.resolve_model_data_config(self.model)
        self.transforms = timm.data.create_transform(**data_config, is_training=False)

    @functools.lru_cache(maxsize=None)
    def extract(self, img) -> torch.Tensor:
        with torch.no_grad():
            return self.model(self.transforms(img).unsqueeze(0))[0]

timm_model = TIMM()

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Renan\.cache\huggingface\hub\models--timm--vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [76]:
# transforma cada par em uma linha de uma ndarray (é quem criou os CSVs dev_train e dev_val)
def pairs_as_ndarray(pairs):
    result = []
    for i, pair in enumerate(pairs):
        image1, image2, label = pair
        print(f'Progress: {i}/{len(pairs)}         ', end='\r')
        attr1 = timm_model.extract(image1)
        attr2 = timm_model.extract(image2)
        row = np.concatenate((attr1, attr2, [label]))
        result.append(row)
    return np.array(result)

In [77]:
dev_train = pairs_as_ndarray(Preprocess.load_train_pairs(transform=TRANSFORM))
dev_val = pairs_as_ndarray(Preprocess.load_test_pairs(transform=TRANSFORM))
# Load from .npy
# dev_train = np.load('dev_train.npy')
# dev_val = np.load('dev_val.npy')
dev_train.shape, dev_val.shape

((2200, 1025), (1000, 1025))

In [78]:
# Save the arrays
np.save('dev_train.npy', dev_train)
np.save('dev_val.npy', dev_val)

In [79]:
X_train = dev_train[:, :-1]
y_train = dev_train[:, -1]
X_val = dev_val[:, :-1]
y_val = dev_val[:, -1]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((2200, 1024), (2200,), (1000, 1024), (1000,))

In [80]:
### Regressão logística
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': np.logspace(-5, 5, 5),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 1e-05, 'solver': 'newton-cg'}
Best cross-validation score:  0.5913636363636363


In [81]:

logreg_model = LogisticRegression(**grid_search.best_params_)
logreg_model.fit(X_train, y_train)
print('Train accuracy:', accuracy_score(y_train, logreg_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, logreg_model.predict(X_val)))


Train accuracy: 0.6381818181818182
Val accuracy: 0.613


In [82]:

### Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=10, cv=5)
rf_random_search.fit(X_train, y_train)

print("Best parameters: ", rf_random_search.best_params_)
print("Best cross-validation score: ", rf_random_search.best_score_)


Best parameters:  {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20}
Best cross-validation score:  0.6972727272727273


In [83]:

rf_model = RandomForestClassifier(**rf_random_search.best_params_)
rf_model.fit(X_train, y_train)

print('Train accuracy:', accuracy_score(y_train, rf_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, rf_model.predict(X_val)))


Train accuracy: 1.0
Val accuracy: 0.721


In [89]:

### Rede neural
import keras_tuner as kt
import keras

def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=4,
                                                max_value=32,
                                                step=4),
                                   activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=5,
    directory='runs',
    project_name=f'nn-timm-mediumd')

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=40, validation_data=(X_val, y_val))

tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(num_trials = 10)[0]


Trial 5 Complete [00h 00m 27s]
val_accuracy: 0.7890000104904175

Best val_accuracy So Far: 0.7895999908447265
Total elapsed time: 00h 02m 29s
Results summary
Results in runs\nn-timm-mediumd
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 3
units_0: 32
learning_rate: 0.01
units_1: 4
units_2: 4
Score: 0.7895999908447265

Trial 4 summary
Hyperparameters:
num_layers: 1
units_0: 20
learning_rate: 0.01
units_1: 8
units_2: 32
Score: 0.7890000104904175

Trial 3 summary
Hyperparameters:
num_layers: 1
units_0: 24
learning_rate: 0.0001
units_1: 24
units_2: 32
Score: 0.7769999861717224

Trial 1 summary
Hyperparameters:
num_layers: 1
units_0: 12
learning_rate: 0.0001
units_1: 20
units_2: 20
Score: 0.7613999962806701

Trial 2 summary
Hyperparameters:
num_layers: 3
units_0: 12
learning_rate: 0.0001
units_1: 8
units_2: 32
Score: 0.7428000092506408


In [94]:

best_hp = tuner.get_best_hyperparameters()[0]
nn_model = tuner.hypermodel.build(best_hp)
nn_model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
nn_model.summary()


Epoch 1/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6016 - loss: 0.6555 - val_accuracy: 0.7410 - val_loss: 0.5248
Epoch 2/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8060 - loss: 0.4325 - val_accuracy: 0.7630 - val_loss: 0.4970
Epoch 3/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8748 - loss: 0.2999 - val_accuracy: 0.7730 - val_loss: 0.5630
Epoch 4/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9029 - loss: 0.2459 - val_accuracy: 0.7740 - val_loss: 0.6833
Epoch 5/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9437 - loss: 0.1429 - val_accuracy: 0.7730 - val_loss: 0.7008
Epoch 6/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9318 - loss: 0.1857 - val_accuracy: 0.7720 - val_loss: 0.8254
Epoch 7/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9356 - loss: 0.1371 - val_accuracy: 0.7670 - val_loss: 0.7946
Epoch 8/40
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9650 - loss: 0.1047 - val_accuracy: 0.7650 - val_loss:

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 32)             │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 4)              │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,873 (386.23 KB)

 Trainable params: 32,957 (128.74 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 65,916 (257.49 KB)

In [95]:

train_accuracy = nn_model.evaluate(X_train, y_train)[1]
val_accuracy = nn_model.evaluate(X_val, y_val)[1]
train_accuracy, val_accuracy


69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 1.0000 - loss: 1.7695e-04
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - accuracy: 0.7719 - loss: 1.7912


(1.0, 0.7829999923706055)

In [87]:


### SVM
from sklearn.svm import SVC

# Define the parameter grid for grid search
param_grid = {
    'C': np.logspace(-3, 2, base=10, num=6),
    'kernel': ['rbf', 'sigmoid'],
    'gamma': np.logspace(-3, 2, base=10, num=6)
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score:  0.7863636363636364


In [88]:

# Create an SVM model with the best parameters
svm_model = SVC(**grid_search.best_params_)
svm_model.fit(X_train, y_train)

# Evaluate the model
print('Train accuracy:', accuracy_score(y_train, svm_model.predict(X_train)))
print('Val accuracy:', accuracy_score(y_val, svm_model.predict(X_val)))

Train accuracy: 1.0
Val accuracy: 0.795
